In [1]:
from time import time
import random
import statistics

In [2]:
epochs_duration = []
epochs_duration.append(time())

### Library heuristic

In [3]:
def library_heuristic(library, books_scores):
    if len(library[1]) == 1:
        return library[1][0]
    books = library[1]
    scores = [books_scores[x] for x in books]
    total_score = sum(scores)
    variance = statistics.variance(scores)
    return total_score / max(0.01, variance)

### Evaluation function 

In [4]:
def evaluate_solution(solution, books_scores):
    books_scanned = set()
    points = 0
    number_of_libraries = solution[0]
    for i in range(1, number_of_libraries+1):
        number_of_books = solution[i][1]
        for j in range(number_of_books):
            book_to_add = solution[i][2][j]
            if book_to_add not in books_scanned:
                books_scanned.add(book_to_add)
    solution_score = 0
    for i in books_scanned:
        solution_score += books_scores[i]
        
    return solution_score

### Get complete solution from list of solutions

In [5]:
def get_complete_solution(solution_indices, libraries, days_number, books_scores):
    time_pointer = days_number
    used_books = set()
    complete_solution = [len(solution_indices)]
    for i in solution_indices:
        time_pointer -= libraries[i][0][1]
        books_per_day = libraries[i][0][2]
        available_books = list(libraries[i][1])
        particular_solution = [i, 0, []]
        available_books.sort(key = lambda x: books_scores[x])
        for x in available_books:
            if len(particular_solution[2]) > time_pointer * books_per_day:
                break
            if x not in used_books:
                particular_solution[2].append(x)
                particular_solution[1] += 1
                used_books.add(x)
        particular_solution[2] = tuple(particular_solution[2])
        complete_solution.append(tuple(particular_solution))
    return tuple(complete_solution)

### Get greedy-like solution

In [6]:
def get_greedy_solution(libraries_number, days_number, libraries, libraries_heuristics, books_scores):
    libraries_to_use = list(range(libraries_number))
    libraries_to_use.sort(key = lambda x: library_heuristic(libraries[x], books_scores), reverse = True)
    solution_indices = []
    time_pointer = 0
    for i in range(libraries_number):
        if time_pointer + libraries[libraries_to_use[i]][0][1] >= days_number:
            break
        solution_indices.append(libraries_to_use[i])
        time_pointer += libraries[libraries_to_use[i]][0][1]
    return get_complete_solution(solution_indices, libraries, days_number, books_scores)

### Initial population

In [7]:
def get_random_solution(libraries_number, days_number, libraries, libraries_heuristics, books_scores):
    libraries_to_use = list(range(libraries_number))
    libraries_to_use_heuristics = list(libraries_heuristics)
    solution_indices = []
    time_pointer = 0
    while True:
        if not len(libraries_to_use):
            return get_complete_solution(solution_indices, libraries, days_number, books_scores)
        random_index = random.choices(list(range(0, len(libraries_to_use))), weights = libraries_to_use_heuristics, k = 1)[0]
        del libraries_to_use[random_index]
        del libraries_to_use_heuristics[random_index]
        if time_pointer + libraries[random_index][0][1] > days_number:
            break                                 
        solution_indices.append(random_index)
        time_pointer += libraries[random_index][0][1]
    return get_complete_solution(solution_indices, libraries, days_number, books_scores)
        
    
def get_initial_population(population_size, libraries_number, days_number, libraries, libraries_heuristics, books, timestamp, max_duration):
    population = []
    for i in range(population_size):
        population.append(get_random_solution(libraries_number, days_number, libraries, libraries_heuristics, books))
        if i % 2 and time() - timestamp >= max_duration / 2:
            return (i, population)
    return (population_size, population)

### Mutation

In [8]:
def mutate(solution, libraries, days_number, books_scores):
    solution_indices = [solution[i][0] for i in range(1, len(solution))]
    for _ in range(3):
        nucleotide_a = random.randint(0, len(solution_indices) - 1)
        nucleotide_b = random.randint(0, len(solution_indices) - 1)
        solution_indices[nucleotide_a], solution_indices[nucleotide_b] = solution_indices[nucleotide_b], solution_indices[nucleotide_a]
    return get_complete_solution(solution_indices, libraries, days_number, books_scores)

def do_mutations(population, libraries, days_number, books_scores):
    mutated_solutions = []
    for solution in population:
        mutated_solutions.append(mutate(solution, libraries, days_number, books_scores))
    return mutated_solutions

## Crossover

In [9]:
def get_children(parent_a, parent_b, libraries, days_number, books_scores):
    parent_a_indices = [parent_a[i][0] for i in range(1, len(parent_a))]
    parent_b_indices = [parent_b[i][0] for i in range(1, len(parent_b))]
    split_point = random.randint(0, min(len(parent_a_indices), len(parent_b_indices)) - 1)
    child_a_indices = parent_a_indices[:split_point] + [x for x in parent_b_indices[split_point:] if x not in parent_a_indices[:split_point]]
    child_b_indices = parent_b_indices[:split_point] + [x for x in parent_a_indices[split_point:] if x not in parent_b_indices[:split_point]]
    for parent in (parent_a_indices, parent_b_indices):
        time_pointer = 0
        for i in range(len(parent)):
            if time_pointer >= days_number:
                parent_a_indices = parent[:i]
                break
            time_pointer += libraries[parent[i]][1]
        time_pointer = 0
    child_a = get_complete_solution(child_a_indices, libraries, days_number, books_scores)
    child_b = get_complete_solution(child_b_indices, libraries, days_number, books_scores)
    return (child_a, child_b)

In [10]:
def get_offspring(population, libraries, days_number, books_scores):
    offspring = []
    mating_pool = [evaluate_solution(population[i], books_scores) for i in range(len(population))] 
    for i in range(len(population), 2):
        offspring.append(get_children(random.choices(population, weights = mating_pool, k = 2)), libraries, days_number, books_scores)
    return tuple(offspring)

# Main

<font color="blue">__Specify how long program is able to run here (in seconds)__</blue>

In [11]:
max_duration = 300

<font color="blue">__Specify population size here__</font>

In [12]:
population_size = 80

### Read data

<font color=blue>__Specify path to input file here__</font>

In [13]:
file = open("instances/a_example.txt", "r")

Load data from source file to the program

In [14]:
books_number, libraries_number, days_number = [int(x) for x in file.readline().split()]
books_scores = tuple([int(x) for x in file.readline().split()])
libraries = [None] * libraries_number
for i in range(libraries_number):
    libraries[i] = (tuple([int(x) for x in file.readline().split()]), tuple([int(x) for x in file.readline().split()]))
libraries = tuple(libraries) 

Get heuristic values for libraries

In [15]:
libraries_heuristics = tuple([library_heuristic(library, books_scores) for library in libraries])

Get initial population

In [16]:
population_size, population = get_initial_population(population_size - 1, libraries_number, days_number, libraries, libraries_heuristics, books_scores, epochs_duration[0], max_duration)

In [17]:
population_size += 1
population.append(get_greedy_solution(libraries_number, days_number, libraries, libraries_heuristics, books_scores))

main loop - epochs <br>


In [18]:
epochs_duration.append(time())
epochs_duration.append(time())
epoch = 0
while epochs_duration[0] + max_duration - 5 >= time() + (epochs_duration[-1] - epochs_duration[-2]) * 2:
    mutations = do_mutations(population, libraries, days_number, books_scores)
    offspring = get_offspring(population, days_number, libraries, books_scores)
    population.extend(mutations)
    population.extend(offspring)
    population.sort(key = lambda x: evaluate_solution(x, books_scores), reverse = True)
    population = population[:population_size]
    epochs_duration.append(time())
    print("epoch: " + str(epoch) + "; timestamp: " + str(round(time() - epochs_duration[0], 2)) +  "; epoch's duration: ", str(round(epochs_duration[-1] - epochs_duration[-2], 2)) + "; best solution: " + str(evaluate_solution(population[0], books_scores)))
    epoch += 1

epoch: 0; timestamp: 0.31; epoch's duration:  0.01; best solution: 21
epoch: 1; timestamp: 0.31; epoch's duration:  0.01; best solution: 21
epoch: 2; timestamp: 0.32; epoch's duration:  0.01; best solution: 21
epoch: 3; timestamp: 0.33; epoch's duration:  0.0; best solution: 21
epoch: 4; timestamp: 0.33; epoch's duration:  0.0; best solution: 21
epoch: 5; timestamp: 0.34; epoch's duration:  0.01; best solution: 21
epoch: 6; timestamp: 0.34; epoch's duration:  0.0; best solution: 21
epoch: 7; timestamp: 0.35; epoch's duration:  0.0; best solution: 21
epoch: 8; timestamp: 0.35; epoch's duration:  0.0; best solution: 21
epoch: 9; timestamp: 0.36; epoch's duration:  0.01; best solution: 21
epoch: 10; timestamp: 0.36; epoch's duration:  0.0; best solution: 21
epoch: 11; timestamp: 0.36; epoch's duration:  0.0; best solution: 21
epoch: 12; timestamp: 0.37; epoch's duration:  0.01; best solution: 21
epoch: 13; timestamp: 0.37; epoch's duration:  0.0; best solution: 21
epoch: 14; timestamp: 0.

epoch: 122; timestamp: 0.95; epoch's duration:  0.01; best solution: 21
epoch: 123; timestamp: 0.96; epoch's duration:  0.0; best solution: 21
epoch: 124; timestamp: 0.96; epoch's duration:  0.0; best solution: 21
epoch: 125; timestamp: 0.96; epoch's duration:  0.0; best solution: 21
epoch: 126; timestamp: 0.97; epoch's duration:  0.0; best solution: 21
epoch: 127; timestamp: 0.97; epoch's duration:  0.0; best solution: 21
epoch: 128; timestamp: 0.98; epoch's duration:  0.0; best solution: 21
epoch: 129; timestamp: 0.98; epoch's duration:  0.0; best solution: 21
epoch: 130; timestamp: 0.98; epoch's duration:  0.0; best solution: 21
epoch: 131; timestamp: 0.99; epoch's duration:  0.0; best solution: 21
epoch: 132; timestamp: 0.99; epoch's duration:  0.0; best solution: 21
epoch: 133; timestamp: 1.0; epoch's duration:  0.0; best solution: 21
epoch: 134; timestamp: 1.0; epoch's duration:  0.01; best solution: 21
epoch: 135; timestamp: 1.01; epoch's duration:  0.01; best solution: 21
epoch

epoch: 290; timestamp: 1.72; epoch's duration:  0.0; best solution: 21
epoch: 291; timestamp: 1.73; epoch's duration:  0.0; best solution: 21
epoch: 292; timestamp: 1.73; epoch's duration:  0.0; best solution: 21
epoch: 293; timestamp: 1.73; epoch's duration:  0.0; best solution: 21
epoch: 294; timestamp: 1.74; epoch's duration:  0.0; best solution: 21
epoch: 295; timestamp: 1.75; epoch's duration:  0.01; best solution: 21
epoch: 296; timestamp: 1.75; epoch's duration:  0.01; best solution: 21
epoch: 297; timestamp: 1.76; epoch's duration:  0.01; best solution: 21
epoch: 298; timestamp: 1.76; epoch's duration:  0.01; best solution: 21
epoch: 299; timestamp: 1.77; epoch's duration:  0.0; best solution: 21
epoch: 300; timestamp: 1.77; epoch's duration:  0.0; best solution: 21
epoch: 301; timestamp: 1.78; epoch's duration:  0.01; best solution: 21
epoch: 302; timestamp: 1.78; epoch's duration:  0.01; best solution: 21
epoch: 303; timestamp: 1.79; epoch's duration:  0.0; best solution: 21


epoch: 431; timestamp: 2.38; epoch's duration:  0.01; best solution: 21
epoch: 432; timestamp: 2.38; epoch's duration:  0.0; best solution: 21
epoch: 433; timestamp: 2.38; epoch's duration:  0.0; best solution: 21
epoch: 434; timestamp: 2.39; epoch's duration:  0.0; best solution: 21
epoch: 435; timestamp: 2.39; epoch's duration:  0.0; best solution: 21
epoch: 436; timestamp: 2.39; epoch's duration:  0.0; best solution: 21
epoch: 437; timestamp: 2.4; epoch's duration:  0.0; best solution: 21
epoch: 438; timestamp: 2.4; epoch's duration:  0.0; best solution: 21
epoch: 439; timestamp: 2.4; epoch's duration:  0.0; best solution: 21
epoch: 440; timestamp: 2.4; epoch's duration:  0.0; best solution: 21
epoch: 441; timestamp: 2.41; epoch's duration:  0.0; best solution: 21
epoch: 442; timestamp: 2.41; epoch's duration:  0.0; best solution: 21
epoch: 443; timestamp: 2.42; epoch's duration:  0.0; best solution: 21
epoch: 444; timestamp: 2.42; epoch's duration:  0.0; best solution: 21
epoch: 44

epoch: 585; timestamp: 3.05; epoch's duration:  0.0; best solution: 21
epoch: 586; timestamp: 3.06; epoch's duration:  0.0; best solution: 21
epoch: 587; timestamp: 3.06; epoch's duration:  0.0; best solution: 21
epoch: 588; timestamp: 3.06; epoch's duration:  0.0; best solution: 21
epoch: 589; timestamp: 3.07; epoch's duration:  0.0; best solution: 21
epoch: 590; timestamp: 3.07; epoch's duration:  0.0; best solution: 21
epoch: 591; timestamp: 3.07; epoch's duration:  0.0; best solution: 21
epoch: 592; timestamp: 3.08; epoch's duration:  0.0; best solution: 21
epoch: 593; timestamp: 3.08; epoch's duration:  0.0; best solution: 21
epoch: 594; timestamp: 3.09; epoch's duration:  0.01; best solution: 21
epoch: 595; timestamp: 3.1; epoch's duration:  0.0; best solution: 21
epoch: 596; timestamp: 3.1; epoch's duration:  0.0; best solution: 21
epoch: 597; timestamp: 3.1; epoch's duration:  0.0; best solution: 21
epoch: 598; timestamp: 3.11; epoch's duration:  0.0; best solution: 21
epoch: 5

...

epoch: 23113; timestamp: 120.47; epoch's duration:  0.01; best solution: 21
epoch: 23114; timestamp: 120.48; epoch's duration:  0.01; best solution: 21
epoch: 23115; timestamp: 120.48; epoch's duration:  0.01; best solution: 21
epoch: 23116; timestamp: 120.49; epoch's duration:  0.01; best solution: 21
epoch: 23117; timestamp: 120.5; epoch's duration:  0.01; best solution: 21
epoch: 23118; timestamp: 120.51; epoch's duration:  0.01; best solution: 21
epoch: 23119; timestamp: 120.51; epoch's duration:  0.01; best solution: 21
epoch: 23120; timestamp: 120.52; epoch's duration:  0.01; best solution: 21
epoch: 23121; timestamp: 120.52; epoch's duration:  0.01; best solution: 21
epoch: 23122; timestamp: 120.53; epoch's duration:  0.01; best solution: 21
epoch: 23123; timestamp: 120.54; epoch's duration:  0.01; best solution: 21
epoch: 23124; timestamp: 120.54; epoch's duration:  0.01; best solution: 21
epoch: 23125; timestamp: 120.55; epoch's duration:  0.01; best solution: 21
epoch: 23126;

epoch: 23242; timestamp: 121.12; epoch's duration:  0.0; best solution: 21
epoch: 23243; timestamp: 121.13; epoch's duration:  0.01; best solution: 21
epoch: 23244; timestamp: 121.14; epoch's duration:  0.0; best solution: 21
epoch: 23245; timestamp: 121.14; epoch's duration:  0.0; best solution: 21
epoch: 23246; timestamp: 121.14; epoch's duration:  0.0; best solution: 21
epoch: 23247; timestamp: 121.15; epoch's duration:  0.0; best solution: 21
epoch: 23248; timestamp: 121.15; epoch's duration:  0.0; best solution: 21
epoch: 23249; timestamp: 121.15; epoch's duration:  0.0; best solution: 21
epoch: 23250; timestamp: 121.16; epoch's duration:  0.0; best solution: 21
epoch: 23251; timestamp: 121.16; epoch's duration:  0.0; best solution: 21
epoch: 23252; timestamp: 121.17; epoch's duration:  0.01; best solution: 21
epoch: 23253; timestamp: 121.17; epoch's duration:  0.0; best solution: 21
epoch: 23254; timestamp: 121.18; epoch's duration:  0.0; best solution: 21
epoch: 23255; timestamp

epoch: 23366; timestamp: 121.79; epoch's duration:  0.0; best solution: 21
epoch: 23367; timestamp: 121.8; epoch's duration:  0.01; best solution: 21
epoch: 23368; timestamp: 121.8; epoch's duration:  0.01; best solution: 21
epoch: 23369; timestamp: 121.81; epoch's duration:  0.01; best solution: 21
epoch: 23370; timestamp: 121.81; epoch's duration:  0.01; best solution: 21
epoch: 23371; timestamp: 121.82; epoch's duration:  0.01; best solution: 21
epoch: 23372; timestamp: 121.83; epoch's duration:  0.01; best solution: 21
epoch: 23373; timestamp: 121.83; epoch's duration:  0.01; best solution: 21
epoch: 23374; timestamp: 121.84; epoch's duration:  0.01; best solution: 21
epoch: 23375; timestamp: 121.84; epoch's duration:  0.01; best solution: 21
epoch: 23376; timestamp: 121.85; epoch's duration:  0.01; best solution: 21
epoch: 23377; timestamp: 121.85; epoch's duration:  0.01; best solution: 21
epoch: 23378; timestamp: 121.86; epoch's duration:  0.01; best solution: 21
epoch: 23379; t

epoch: 23475; timestamp: 122.46; epoch's duration:  0.0; best solution: 21
epoch: 23476; timestamp: 122.47; epoch's duration:  0.01; best solution: 21
epoch: 23477; timestamp: 122.47; epoch's duration:  0.0; best solution: 21
epoch: 23478; timestamp: 122.48; epoch's duration:  0.01; best solution: 21
epoch: 23479; timestamp: 122.48; epoch's duration:  0.0; best solution: 21
epoch: 23480; timestamp: 122.49; epoch's duration:  0.0; best solution: 21
epoch: 23481; timestamp: 122.49; epoch's duration:  0.0; best solution: 21
epoch: 23482; timestamp: 122.49; epoch's duration:  0.0; best solution: 21
epoch: 23483; timestamp: 122.5; epoch's duration:  0.0; best solution: 21
epoch: 23484; timestamp: 122.5; epoch's duration:  0.0; best solution: 21
epoch: 23485; timestamp: 122.51; epoch's duration:  0.0; best solution: 21
epoch: 23486; timestamp: 122.51; epoch's duration:  0.0; best solution: 21
epoch: 23487; timestamp: 122.51; epoch's duration:  0.0; best solution: 21
epoch: 23488; timestamp: 

KeyboardInterrupt: 

In [ ]:
print(population[0][0])
for i in range(1, len(population[0])):
    print(population[0][i][0], population[0][i][1], end = " ")
    print(" ".join([str(x) for x in population[0][i][2]]))

In [ ]:
print(evaluate_solution(population[0], books_scores))

In [ ]:
print(time() - epochs_duration[0])